In [1]:
!pip install yfinance

In [5]:
!pip install xlsxwriter

In [17]:
import yfinance as yf
import pandas as pd
import requests

In [182]:
crude_oil_ticker = "CL=F"
exr_eurusd_ticker = "EURUSD=X"
exr_eurrub_ticker = "EURRUB=X"
hist = crude_oil.history()

In [183]:
data = yf.download(crude_oil_ticker)
exr = yf.download(exr_eurusd_ticker)
rub = yf.download(exr_eurrub_ticker)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [184]:
exr

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-12-01,1.203398,1.204007,1.194401,1.196501,1.196501,0
2003-12-02,1.196101,1.210903,1.194600,1.208897,1.208897,0
2003-12-03,1.209000,1.213003,1.207700,1.212298,1.212298,0
2003-12-04,1.212004,1.214403,1.204398,1.208094,1.208094,0
2003-12-05,1.207802,1.219096,1.206593,1.218695,1.218695,0
...,...,...,...,...,...,...
2023-04-03,1.080334,1.091572,1.078935,1.080334,1.080334,0
2023-04-04,1.091227,1.097165,1.088388,1.091227,1.091227,0
2023-04-05,1.096011,1.096924,1.090536,1.096011,1.096011,0


# Задание 1.

* Вы - менеджер по продажам в нефтехимической компании. Вы ведете несколько ключевых клиентов, которые закупают продукты вашей компании. Ваша компания запускает новый продукт - "Синтетический каучук" (СК), который по своим свойствам похож на тот, что уже есть на рынке. Ваши клиенты предпочитают формульный подход к ценообразованию. Вам нужно предложить клиентам формулу расчета цены на СК. В зависимости от объемов, которые покупают клиенты, им может быть предложена скидка.


* СК производиться из нефти. Один из известных способов производства предполагает, что из 10 баррелей нефти сделают одну тонну СК, а затраты на производство в районе 400 евро.

In [14]:

# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
customers = {
    'Сonty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Ant':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
discounts = {'99': 0.01, # до 100 тонн 1%
             '299': 0.05, #  до 300 тонн 5%
             '300': 0.1}   # больше 300 тонн 10%


### Что нужно сделать?

1. Определить базовую формулу цены на заводе. То есть как бы выглядела цена на исторических данных. 
* Важно, чтобы цена покрывала расходы на производство и при этом была в рынке, то есть на уровне натурального каучука или с небольшим дисконтом, котировки на который можно взять с сайта http://www3.lgm.gov.my/mre/MonthlyPrices.aspx (в центах за килограм). Можно перевести в доллары за тонну.
2. Отобразить на графике.
3. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
4. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
- Создать директорию «для клиентов» и в ней сложить  файлы с расчетами.
- Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf)  с именем клиента… 



In [195]:
# Котировки натурального каучука доступны на сайте,  https://www.lgm.gov.my
# также у них есть API, пример запроса ниже
#
year = '2023'
month = '03'

grade = url = f"https://www.lgm.gov.my/webv2api/api/rubberprice/month={month}&year={year}"
res = requests.get(url)
rj = res.json()
df2 = pd.json_normalize(rj)
df2.head(10)

,date,grade,masa,rm,us,tone
0,2023-03-31,SMR CV,Noon,866.00,199.45,Quietly Steady
1,2023-03-31,SMR L,Noon,856.00,197.15,Quietly Steady
2,2023-03-31,SMR 5,Noon,605.00,139.35,Quietly Steady
3,2023-03-31,SMR GP,Noon,602.00,138.65,Quietly Steady
4,2023-03-31,SMR 10,Noon,595.00,137.05,Quietly Steady
5,2023-03-31,SMR 20,Noon,593.00,136.55,Quietly Steady
6,2023-03-31,Latex in Bulk,Noon,507.50,507.50,Quietly Steady
7,2023-03-30,SMR CV,Noon,870.00,199.85,Quietly Steady
8,2023-03-30,SMR L,Noon,860.00,197.55,Quietly Steady
9,2023-03-30,SMR 5,Noon,603.50,138.65,Quietly Steady


In [196]:
df2['us'] = pd.to_numeric(df2['us'], errors='coerce')
df2.dropna(inplace=True)

In [197]:
df2['trunc'] = pd.to_datetime(df2['date']).dt.to_period('M')

In [201]:
df2 = df2[df2['grade']=='SMR 10']

In [202]:
result = df2.groupby('trunc')['us'].mean()
df_now = result.to_frame(name='average_us') * 10


In [203]:
df_now

,average_us
trunc,
2023-03,1354.804348


In [205]:
df = pd.concat([data.resample('M').mean()['Close'], exr.resample('M').mean()['Close']], axis=1)
df.columns = ['crude_oil_price', 'eureusd']
df = df['2022-01-01':'2022-12-31']
# # Formula (10*Data + 400 ) * Discount + Logistics
df["proccessing_usd"] = df['eureusd'] * 400
df["base_wpb_price_usd"] = df["crude_oil_price"]*10 + df['proccessing_usd']

In [133]:
df

,crude_oil_price,eureusd,proccessing_usd,base_wpb_price_usd
Date,,,,
2022-01-31,82.979000,1.132574,453.029503,1282.819504
2022-02-28,91.628947,1.133995,453.598197,1369.887665
2022-03-31,108.262174,1.102412,440.964999,1523.586738
2022-04-30,101.640499,1.082745,433.097801,1449.502796
2022-05-31,109.259523,1.057322,422.928960,1515.524189
2022-06-30,114.154091,1.057670,423.067819,1564.608731
2022-07-31,99.384500,1.019509,407.803622,1401.648619
2022-08-31,91.482609,1.012799,405.119594,1319.945681
2022-09-30,83.801905,0.991166,396.466519,1234.485568


In [206]:
def check_volume_discounts(x):
  if 0 < x <= 99:
    return 0.01
  elif 99 < x <= 299:
    return 0.05
  elif x >= 300:
    return 0.1
  else:
    return 0

In [207]:
def price_now(x,y,z,w):
    new = x * (1-y) + z
    if new > w:
        return new
    else: 
        return w

In [214]:
xlfilepath = 'wbp_prices.xlsx'
with pd.ExcelWriter(xlfilepath, engine='xlsxwriter') as writer:
  for client_name, client_info in customers.items():
    client_df = df.copy()
    
    if client_info.get('location') == 'EU':
      client_df['logistics'] = client_df['eureusd'] * EU_LOGISTIC_COST_EUR
    elif client_info.get('location') == 'CN':
      client_df['logistics'] = CN_LOGISTIC_COST_USD
    
    if client_info.get('volumes'):
      client_df['volumes'] = client_info.get('volumes')

    client_df['discount'] = client_df['volumes'].apply(check_volume_discounts)
    average_us = df_now['average_us'][0]
    client_df['WBP_price_usd'] = client_df.apply(lambda x: price_now(x['base_wpb_price_usd'],x['discount'],x['logistics'],average_us), axis=1)
    
    client_df['WBP_price_usd'].plot(color='red', linestyle="dashed")
    plt.title(f"Client {client_name} WBP Price")
    plt.ylabel("USD")
    plt.tight_layout()
    plt.savefig(f'{client_name}_wbp_price.png')
    plt.close()

    client_df = client_df.round(2)
    client_df = client_df.reset_index()
    client_df.Date = client_df.Date.dt.strftime('%B %Y')
    
    max_row, max_col = client_df.shape
    client_df.to_excel(writer, sheet_name=client_name, startrow=1, header=False, index=False)
    workbook = writer.book
    worksheet = writer.sheets[client_name]
    column_settings = [{'header': column} for column in client_df.columns]
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
    worksheet.insert_image(max_row + 3, 1, f'{client_name}_wbp_price.png')
    

# Задание 2

Клиенты согласились на ваше предложение. 
Вам нужно сделать написать скрипт, с помощью которого можно автоматом рассчитывать цены для всех клиентов одновременно. 
Важный момент: данные для расчета скрипт должен подтягивать сам.

In [215]:
clients_path = 'for_clients'
os.makedirs(clients_path, exist_ok=True)

for client_name, client_info in customers.items():
    client_df = df.copy()
    
    if client_info.get('location') == 'EU':
      client_df['logistics'] = client_df['eureusd'] * EU_LOGISTIC_COST_EUR
    elif client_info.get('location') == 'CN':
      client_df['logistics'] = CN_LOGISTIC_COST_USD
    
    if client_info.get('volumes'):
      client_df['volumes'] = client_info.get('volumes')

    client_df['discount'] = client_df['volumes'].apply(check_volume_discounts)
    average_us = df_now['average_us'][0]
    client_df['WBP_price_usd'] = client_df.apply(lambda x: price_now(x['base_wpb_price_usd'],x['discount'],x['logistics'],average_us), axis=1)
    
    client_df['WBP_price_usd'].plot(color='red', linestyle="dashed")
    plt.title(f"Client {client_name} WBP Price")
    plt.ylabel("USD")
    plt.tight_layout()
    plt.savefig(f'{client_name}_wbp_price.png')
    plt.close()

    client_df = client_df.round(2)
    client_df = client_df.reset_index()
    client_df.Date = client_df.Date.dt.strftime('%B %Y')
    
    max_row, max_col = client_df.shape
    
    xlfilepath = os.path.join(clients_path, f'{client_name}_wbp_prices.xlsx')
    with pd.ExcelWriter(xlfilepath, engine='xlsxwriter') as writer:
        client_df.to_excel(writer, sheet_name=client_name, startrow=1, header=False, index=False)
        workbook = writer.book
        worksheet = writer.sheets[client_name]
        column_settings = [{'header': column} for column in client_df.columns]
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
        worksheet.insert_image(max_row + 3, 1, f'{client_name}_wbp_price.png')

# Задание 3

К вам пришел новый клиент из России. Который использует продукт А и готов регулярно закупать СК для того, чтобы не быть привязанным к одному поставщику. Клиент готов забирать СК с завода самостоятельно.
На основе тех данных, которые уже есть предложите цену новому клиенту в рублях. 



In [185]:
df = pd.concat([data.resample('M').mean()['Close'], rub.resample('M').mean()['Close']], axis=1)
df.columns = ['crude_oil_price', 'eurrub']
df = df['2022-01-01':'2022-12-31']
# # Formula (10*Data + 400 ) * Discount
df["proccessing_rub"] = df['eurrub'] * 400
df["base_wpb_price_usd"] = df["crude_oil_price"]*10 + df['proccessing_rub']